# Deep Learning
## Exercise 8 - Unsupervised Approaches


### 1. Autoencoder
Your task is to train an autoencoder on the [FashionMNIST dataset](https://github.com/zalandoresearch/fashion-mnist) containing grayscale Zalando article images of 10 different categories. The data setup is already provided and should feel familiar from previous assignments.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

train_dataset = torchvision.datasets.FashionMNIST(root="./data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
eval_dataset = torchvision.datasets.FashionMNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor(), download=True)

batch_size = 1024

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, num_workers=2)

def show_examples(n):
    for i in range(n):
        index = torch.randint(0, len(train_dataset), size=(1,)).item() # select a random example
        image, target = train_dataset[index]
        print(f'image of shape: {image.shape}')
        print(f'label: {train_dataset.classes[target]}')
        plt.imshow(image.squeeze().numpy(), cmap='gray')
        plt.show()

show_examples(4)

#### 1. Implement an Autoencoder

The autoencoder shoud have the following architecture:
* encoder: flatten the input, then apply two fully connected layers, one with 256 hidden neurons and the second one with 64 hidden neurons. Apply a ReLU after every fully connected layer.
* decoder: symmetric to the encoder layers.

In [ ]:
#ToDo: Implement the encoder and the decoder

In [ ]:
encoder = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28*28, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=64),
    nn.ReLU()
    )

decoder = nn.Sequential(
    nn.Linear(in_features=64, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=28*28),
    nn.ReLU(),
    nn.Unflatten(1, (1,28,28))
    )

#### 2. Train the network for 20 epochs

Use a learning rate of 0.001 and the Adam optimizer, using a mean-squared-error (MSE) input reconstruction loss. For each epoch, compute the train loss and eval loss (average over batches).

Try to visualize some of the reconstructed images, for example by creating a modified version of the given `show_examples` function that additionaly plots the reconstructed image.

In [ ]:
#ToDo: Train the network

In [ ]:
from tqdm import tqdm

def train(epochs, encoder, decoder, optimizer, loss_function, train_dl, val_dl):
    for epoch in range(epochs):
        encoder.train()
        decoder.train()
        cum_train_loss = 0
        for img, target in tqdm(train_dl, desc='Train Iteration', ascii=True):
            rec_img = decoder(encoder(img))
            loss = loss_function(rec_img, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            cum_train_loss += loss.item()
        encoder.eval()
        decoder.eval()
        cum_val_loss = 0
        with torch.no_grad():
            for img, target in tqdm(val_dl, desc='Val Iteration', ascii=True):
                rec_img = decoder(encoder(img))
                loss = loss_function(rec_img, img)
                cum_val_loss += loss.item()
        print(f"Epoch {epoch} \t Train Loss {cum_train_loss/len(train_dl)} \t Val Loss {cum_val_loss/len(val_dl)}")
        if epoch%5==0:
            show_examples(2, encoder, decoder)
    show_examples(5, encoder, decoder)
                
def show_examples(n, encoder, decoder):
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        for i in range(n):
            fig, (ax1, ax2) = plt.subplots(1,2)
            index = torch.randint(0, len(eval_dataset), size=(1,)).item() # select a random example
            image, target = train_dataset[index]
            print(f'image of shape: {image.shape}')
            print(f'label: {train_dataset.classes[target]}')
            ax1.imshow(image.squeeze().numpy(), cmap='gray')
            rec_image = decoder(encoder(image))
            ax2.imshow(rec_image.squeeze().numpy(), cmap='gray')
            plt.show()
            
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=0.001)
loss_function = nn.MSELoss()
train(20, encoder, decoder, optimizer, loss_function, train_loader, eval_loader)

### 2. GANs
Train a Generative Adversarial Network to generate handwritten digits on the MNIST dataset. Again, the data setup is already given. The images are of size (1x28x28)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision as tv
import matplotlib.pyplot as plt

batch_size = 64

tfms = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = tv.datasets.MNIST(root='./data', train=True, transform=tfms, download=True)
classes = train_dataset.classes
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

def show_examples(n):
    for i in range(n):
        index = torch.randint(0, len(train_dataset), size=(1,)).item() # select a random example
        print(index)
        image, target = train_dataset[index]
        print(f'image of shape: {image.shape}')
        print(f'label: {classes[target]}')
        plt.imshow(image.squeeze().numpy(), cmap='gray')
        plt.show()

show_examples(4)

#### 1. Build a GAN

Your GAN should be built using the following components:
- A Generator that
    - as input, uses $100$-dim latent vector sampled from a normal distribution around mean $=0$ and std $=1$.
    - has 5 fully connected layers, with dimensions $128 \rightarrow 256 \rightarrow 512 \rightarrow 1024 \rightarrow 28\cdot 28$
    - applies 1d batch normalization after the first 4 layers
    - applies a LeakyReLU with a negative slope of $0.2$ after the first 4 layers (after  batchnorm)
    - uses a Tanh activation after the last layer, to generate pixel values between -1 and 1.
- A Discriminator that
    - as input, takes an image of flattened size $28 \cdot 28$
    - has 3 fully connected layers, with dimensions $28 \cdot 28 \rightarrow 512 \rightarrow 256 \rightarrow 1$
    - applies a LeakyReLU with a negative slope of $0.2$ after the first 2 layers
    - uses a sigmoid activation after the last layer, to output probabilities between 0 and 1.

- Two optimizers, one for the generator and one for the discriminator. Use Adam with a learning rate of $0.001$ and `betas=(.5, .999)`.
- Binary Cross Entropy loss

In [ ]:
#ToDo: Set up your GAN

In [ ]:
def get_latent_vectors(batch_size, generator):
    return torch.randn((batch_size, 100), generator=generator)

Generator=nn.Sequential(
    nn.Linear(in_features=100, out_features=128),
    nn.BatchNorm1d(128),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=128, out_features=256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=256, out_features=512),
    nn.BatchNorm1d(512),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=512, out_features=1024),
    nn.BatchNorm1d(1024),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=1024, out_features=28*28),
    nn.Tanh(),
    nn.Unflatten(1,(1,28,28))
)

Discriminator=nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=28*28, out_features=512),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=512, out_features=256),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Linear(in_features=256, out_features=1),
    nn.Sigmoid(),
)

opt_gen = torch.optim.Adam(Generator.parameters(), lr=0.001, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(Discriminator.parameters(), lr=0.001, betas=(0.5, 0.999))
loss_function = torch.nn.BCELoss()

Generator.eval()
Discriminator.eval()
rand_gen = torch.Generator().manual_seed(0)
torch.manual_seed(0)
gen_img=Generator(get_latent_vectors(1, rand_gen))
plt.imshow(gen_img.detach().squeeze(), cmap='gray')
plt.show()
print(Discriminator(gen_img))

#### 2. Train your GAN for multiple epochs

For every batch you should:
- use a vector of zeros as your target vector indicating fake images
- use a vector of ones as your target vector indicating real images
- train the generator by 
    - sampling a random latent vector
    - generate a batch of fake images using the generator
    - compute the loss using the discriminators predictions. As groundtruth, use the label that indicates the images are real.
- train the discriminator by
    - sampling a random latent vector
    - generate a batch of fake images using the generator
    - computing the loss for both the generated fake images and the real images from the batch, using the corresponding target vectors.

Try to visualize some generated images every few epochs. 

Note: GANs are difficult to train. Don't worry if your results aren't perfect.

In [ ]:
#ToDo: Train the GAN

In [ ]:
from tqdm import tqdm

def train_gen(Gen, Disc, opt_gen, loss_function, img_batch, rand_gen):
    latent_vectors = get_latent_vectors(img_batch.shape[0], rand_gen)
            
    fake_img = Gen(latent_vectors)

    real_target = torch.ones((img_batch.shape[0]))

    gen_loss = loss_function(Disc(fake_img).flatten(), real_target)

    opt_gen.zero_grad()
    gen_loss.backward(retain_graph=True)
    opt_gen.step()
    return gen_loss.item()

def train_disc(Gen, Disc, opt_disc, loss_function, img_batch, rand_gen):
    latent_vectors = get_latent_vectors(img_batch.shape[0], rand_gen)
            
    fake_img = Gen(latent_vectors)

    real_target = torch.ones((img_batch.shape[0]))
    fake_target = torch.zeros((img_batch.shape[0]))
    opt_disc.zero_grad()

    real_loss = loss_function(Disc(img_batch).flatten(), real_target)
    fake_loss = loss_function(Disc(fake_img).flatten(), fake_target)
    disc_loss = (real_loss + fake_loss) / 2
    disc_loss.backward()
    opt_disc.step()
    return disc_loss.item()


def train(num_epochs, Gen, Disc, opt_gen, opt_disc, loss_function, train_dl):
    rand_gen = torch.Generator().manual_seed(0)
    torch.manual_seed(0)
    for epoch in range(num_epochs):
        Gen.train()
        Disc.train()
        Gen_Losses = 0
        Disc_Losses = 0
        for img, target in tqdm(train_dl, ascii=True):
            Gen_Losses += train_gen(Gen, Disc, opt_gen, loss_function, img, rand_gen)
            Disc_Losses += train_disc(Gen, Disc, opt_disc, loss_function, img, rand_gen)
                
        print(f"Epoch {epoch}: Disc {Disc_Losses/len(train_dl)} Gen {Gen_Losses/len(train_dl)}")
        if epoch%5 == 0:
            show_examples(5, Gen)
    show_examples(10, Gen)
        

def show_examples(n, model):
    model.eval()
    with torch.no_grad():
        images = model(get_latent_vectors(n, rand_gen))
        images = torch.cat([img for img in images], 2).squeeze()
    fig = plt.figure(figsize = (n*2,3))
    ax = fig.add_subplot(111)
    ax.imshow(images.detach().numpy(), cmap='gray')
    plt.show()

        
train(20, Generator, Discriminator, opt_gen, opt_disc, loss_function, train_loader)

            